In [1]:
# Python中可视化多波段卫星影像
#！/usr/bin/env python3
# -*- coding:utf-8 -*-

%matplotlib inline

from osgeo import gdal

dem = gdal.Open(r"D:\GoogleDownLoads\CasMVSNet_pl\gdal_study\RSP_testing_data\JAX_IMG1_MSI.TIF")  # gdal.Dataset

gt = dem.GetGeoTransform()

# ================================1.获取原始图像左上角的地理坐标===============================================
xmin = gt[0]+0*gt[1]+0*gt[2]#g[0-2]分别是x方向分辨率
ymin = gt[3]+0*gt[4]+0*gt[5]
res = gt[1]# 影像分辨率，注意x方向为正,y方向分辨率为负，但数值上x,y方向上一致


# ================================2.计算原始图像地理坐标范围长度和宽度========================================
xlen = res * dem.RasterXSize
ylen = res * dem.RasterYSize
print(f'xlen={xlen},ylen={ylen}')

# ================================3.设置x,y方向上的分块数=============================================
xdiv = 2
ydiv = 2


# ================================4.计算单块的尺寸====================================================
xsize = xlen/xdiv
ysize = ylen/ydiv
print(f'xsize={xsize},ysize={ysize}')

# ================================5.计算出分块的角点坐标===============================================
xsteps = [xmin + xsize * i for i in range(xdiv+1)]
print(f'xsteps={xsteps}')
ysteps = [ymin +ysize * i for i in range(ydiv+1)]
print(f'ysteps={ysteps}')

# ================================6.循环进行裁剪===============================================
for i in range(xdiv):
    for j in range(ydiv):
        xmin = xsteps[i]
        xmax = xsteps[i+1]
        ymin = ysteps[j]
        ymax = ysteps[j+1]
        
        print("xmin: "+str(xmin))
        print("xmax: "+str(xmax))
        print("ymin: "+str(ymin))
        print("ymax: "+str(ymax))
        print("\n")
        
        #使用 gdal warp裁剪
        # gdal.Warp("dem"+str(i)+str(j)+".tif", dem, 
        #           outputBounds = (xmin, ymin, xmax, ymax), dstNodata = -9999)
        #或者使用 gdal translate裁剪，但需要注意两个的传入裁剪范围顺序
        gdal.Translate("dem_translate"+str(i)+str(j)+".tif", dem, projWin = (ymin, xmin, ymax,xmax))
 
# 7.关闭
dem = None






xlen=1248.0,ylen=1360.0
xsize=624.0,ysize=680.0
xsteps=[0.0, 624.0, 1248.0]
ysteps=[0.0, 680.0, 1360.0]
xmin: 0.0
xmax: 624.0
ymin: 0.0
ymax: 680.0


xmin: 0.0
xmax: 624.0
ymin: 680.0
ymax: 1360.0


xmin: 624.0
xmax: 1248.0
ymin: 0.0
ymax: 680.0


xmin: 624.0
xmax: 1248.0
ymin: 680.0
ymax: 1360.0




In [2]:
from osgeo import gdal
dem = gdal.Open(r"D:\GoogleDownLoads\CasMVSNet_pl\gdal_study\RSP_testing_data\JAX_IMG1_MSI.TIF")  # gdal.Dataset

#=========================== 1.图像坐标裁剪区域=======================
x=1000
y=500
h=500
w=1000
# 按照图像坐标进行裁剪
gdal.Translate("dem_translateSrc"+".tif", dem, srcWin = (y,x,h,w))
#======================== 2.计算出图像坐标对应的地理坐标=====================
def GeoCoord(dataset, x, y):
    GT = dataset.GetGeoTransform()#(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
    print(GT)
    if GT:
        return [GT[0] + x*GT[1] + y*GT[2], GT[3] + x*GT[4] + y*GT[5]]
# 2.1先计算对应的地理坐标
xmin, ymin=GeoCoord(dem,x,y)#xmin,ymin
print(f'xmin={xmin},ymin={ymin}')
xmax, ymax=GeoCoord(dem,x+w,y+h)
print(f'xmax={xmax},ymax={ymax}')
# 2.2 执行根据地理坐标范围进行裁剪
gdal.Translate("dem_translateProj"+".tif", dem, projWin = (ymin,xmin,ymax,xmax))#左下角和右上角

(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
xmin=1000.0,ymin=500.0
(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
xmax=2000.0,ymax=1000.0


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001A11AD3A3F0> >